**For Fine-tuning**
===
> -*- coding: utf-8 -*-

> Author: xinghd
> Author: liudongliang


## Import libraries

In [1]:
from utils.utils import Trainer, Predictor, TrainReader,TestReader,PredictReader, data_collate, get_shuffle_data, collate_fn, seed_everything
from model.model import Encoder, Decoder, ModelCat

%reload_ext autoreload
%autoreload 2

In [2]:
import os
import json
import random
import time
import timeit
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from torch import nn
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import torch
from torch.utils.data import DataLoader, random_split
from utils.word2vec import seq_to_kmers, get_protein_embedding
from gensim.models import Word2Vec
from torch.utils import data as torch_data
import warnings

### Parameters

In [3]:
assert torch.cuda.is_available(), "Must have a avaliable gpu"

class CFG :
    # train dataset and test dataset
    traindata_path = './fine_tuning/train_random.csv'
    testdata_path = './fine_tuning/test_random.csv'
    # epoch
    EPOCHES = 20
    # batch size
    BATCH_SIZE = 1
    
    # learning rate
    lr = 1e-5
    # weight_decay
    weight_decay = 1e-7
    
    # suffix of log/model filename
    modelsave_file_suffix = 'funetuning_random.pt'
    result_file_suffix = 'funetuning_log_random.txt'
    
    # create log file
    result_file_path = './results/log/funetuning_log/'
    # save_best_model(depends on AUC)
    best_model_savepath = "./model/model_funetuning_random/"

    
    # use wandb(https://wandb.ai/)?
    IFwandb = False
    # data root
    DATA_ROOT = r'./data/'
    # save unlabel data that may be positive
    U_m_P_savepath = r"./data/U_m_P/"
    # word2vec model path
    word2vec_path = './model/model_pretrained/word2vec_pretrained.model'
    # pre_trained model path , if given None, training from scratch
    state_dict_path = './model/model_pretrained/PU-EPP_pretrained.pt'
    # Number CUDA Devices:
    gpu_number = torch.cuda.device_count()
    # DEVICE
    DEVICE = torch.device('cuda:0')

    
    
    # block layers
    layer_num = 12
    # The last dimension of the protein dataapt list --upgradable
    protein_dim = 100
    # The last dimension of the compound data
    atom_dim = 46
    # hidden dimension
    hid_dim = 128
    # norm_shape: layernorm parameter
    norm_shape = 128
    # ====================================================
    # to create trainreader
    # ====================================================
    # if use pu learning
    ifpu = True
    # if use label smoothing
    ifsmoothing = True

    # start deleting data when auc value is greater than del_threshold
    del_threshold=0.9

    quantile = 0.9


## WandB

In [4]:
if CFG.IFwandb:
    import wandb
    wandb.login()
    wandb.init(project='PU-EPP', name='finetuning')

## Parameters

### Random Seed

In [5]:
seed_everything(seed=42)

## Model

In [6]:
encoder = Encoder(CFG.protein_dim, CFG.hid_dim, CFG.norm_shape)
decoder = Decoder(CFG.atom_dim, CFG.hid_dim, CFG.norm_shape)
model = ModelCat(encoder, decoder, atom_dim=CFG.atom_dim)
model = model.to(CFG.DEVICE)
model = nn.DataParallel(model, device_ids = list(range(CFG.gpu_number)))

In [7]:
model.load_state_dict(torch.load(CFG.state_dict_path, map_location=CFG.DEVICE))

<All keys matched successfully>

In [8]:
if CFG.IFwandb:
    wandb.watch(model, log='all')

## Trainer

In [9]:
""" create trainer """
# /utils/builder.py
trainer = Trainer(model, CFG) #trainfirst=False

## Dataset & Dataloader

In [10]:
trainset = pd.read_csv(CFG.traindata_path)
testset = pd.read_csv(CFG.testdata_path)



### Dataloader

In [11]:
traindata = TrainReader(data=trainset,U_m_P_savepath=CFG.U_m_P_savepath,
                                word2vec_path=CFG.word2vec_path, ifpu=CFG.ifpu)
testdata = TestReader(data=testset, word2vec_path=CFG.word2vec_path)

''' if the pre-trained word2vec model does not include all amino acid embedding of the enzymes in your dataset, please re-train the word2vec model on your dataset'''

# train_word2vec(list(traindata['Protein'].unique()) + list(testdata['Protein'].unique()), './model/model_pretrained/word2vec_yourself.model') 

# CFG.word2vec_path = './model/model_pretrained/word2vec_yourself.model' 



' if the pre-trained word2vec model does not include all amino acid embedding of the enzymes in your dataset, please re-train the word2vec model on your dataset'

In [12]:
val_dataloader = DataLoader(testdata, batch_size=CFG.BATCH_SIZE,collate_fn=collate_fn)

## Train

In [13]:
file_model = CFG.modelsave_file_suffix
reshead = 'Epoch\tTime(sec)\tLoss_train\tAUC_dev\tPre\tRecall\tPRC_dev'
file_res = CFG.result_file_path+CFG.result_file_suffix
if os.path.exists(file_res):
    warnings.warn("----------------file name duplicate-----------------")
else:
    with open(file_res, 'w') as f:
        f.write(reshead + '\n')

In [14]:
max_AUC_dev = 0
for epoch in range(CFG.EPOCHES):
    print('epoch: ' + str(epoch))
    train_dataloader = DataLoader(traindata, batch_size=CFG.BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

    """Start training."""
    print('Training........................')
    torch.cuda.empty_cache()
    start = timeit.default_timer()

    loss_train, U_m_P = trainer.train(train_dataloader)
    AUC_dev, precision, recall, PRC_dev = trainer.test(val_dataloader)

    end = timeit.default_timer()
    time = end - start
    
    reslist = [epoch, time, loss_train, AUC_dev, precision, recall, PRC_dev]
    if CFG.IFwandb:
        wandb.log({'loss_train':loss_train, "AUC_dev":AUC_dev, 'pre':precision, 'recall':recall, 'prc':PRC_dev})
    trainer.save_AUCs(reslist, file_res)
    if AUC_dev > max_AUC_dev:
        trainer.save_model(model, f"{CFG.best_model_savepath}_epoch{epoch}_{AUC_dev}.pt")
        max_AUC_dev = AUC_dev
    print('\t'.join(map(str, reslist)))

    if AUC_dev > 0.85: #changed
        print(f"del {epoch}:{len(U_m_P)}")
        traindata.del_U(epoch, U_m_P)
    else:
        traindata.reset_T()
    if epoch % 5 == 0:
        trainer.save_model(model, f"{CFG.best_model_savepath}_epoch{epoch}_{AUC_dev}.pt")
    torch.cuda.empty_cache()

epoch: 0
Training........................
0	52.70385742199869	430.3982539176941	0.7386569872958257	0.5681818181818182	0.6578947368421053	0.717953466719069
epoch: 1
Training........................
1	51.8513990820029	408.2514324784279	0.7114337568058077	0.5652173913043478	0.6842105263157895	0.702075391489943
epoch: 2
Training........................
2	51.28487097799734	387.5357081592083	0.7649727767695099	0.5918367346938775	0.7631578947368421	0.7098820901812924
epoch: 3
Training........................
3	51.62673168699985	376.63840341567993	0.7250453720508166	0.5652173913043478	0.6842105263157895	0.5999417542932529
epoch: 4
Training........................
4	51.64561548900019	368.4258399605751	0.7173321234119782	0.5510204081632653	0.7105263157894737	0.6221122346572264
epoch: 5
Training........................
5	51.73597335299928	364.8502376675606	0.8362068965517242	0.5964912280701754	0.8947368421052632	0.7675347438845863
epoch: 6
Training........................
6	51.70021115599957	372.

In [15]:
max_AUC_dev

0.9183303085299455